In [2]:
import os
import requests

# 1. Define paths and URL
SR_MODEL_DIR = 'sr_models'
SR_MODEL_PATH = os.path.join(SR_MODEL_DIR, 'EDSR_x4.pb')
SR_MODEL_URL = 'https://github.com/opencv/opencv_extra/raw/master/testdata/dnn_superres/EDSR_x4.pb'

# 2. Create the directory if it doesn't exist
os.makedirs(SR_MODEL_DIR, exist_ok=True)

# 3. Check if the file already exists
if os.path.exists(SR_MODEL_PATH):
    print(f"File already exists at {SR_MODEL_PATH}. Skipping download.")
else:
    print(f"Downloading Super-Resolution model from {SR_MODEL_URL}...")
    try:
        # Get the content from the URL
        response = requests.get(SR_MODEL_URL, stream=True)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        # Write the content to the local file
        with open(SR_MODEL_PATH, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        print(f"Download complete. File saved to: {SR_MODEL_PATH}")

    except Exception as e:
        print(f"An error occurred during download. You might need to install 'requests': pip install requests")
        print(f"Error: {e}")

File already exists at sr_models\EDSR_x4.pb. Skipping download.


In [3]:
import cv2

# Check the version (should be 4.x)
print(f"OpenCV Version: {cv2.__version__}")

# This check should now be True
if hasattr(cv2, 'dnn_superres'):
    print("Success! 'dnn_superres' module is available.")
else:
    print("Error: 'dnn_superres' is still not available.")

# You can also try to create the object directly
try:
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    print("Test successful: DnnSuperResImpl_create() ran without error.")
except AttributeError as e:
    print(f"Test failed even after reinstall: {e}")

OpenCV Version: 4.12.0
Success! 'dnn_superres' module is available.
Test successful: DnnSuperResImpl_create() ran without error.


In [4]:
import cv2
import os
from ultralytics import YOLO

def fast_video_yolo_sr(
    input_video,
    output_video,
    yolo_model_path,
    scale=2,
    skip_frames=2
):
    model = YOLO(yolo_model_path)

    cap = cv2.VideoCapture(input_video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(
        output_video,
        cv2.VideoWriter_fourcc(*"mp4v"),
        fps,
        (w * scale, h * scale)
    )

    frame_id = 0
    last_results = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1

        # YOLO only every N frames
        if frame_id % skip_frames == 0:
            last_results = model(frame, verbose=False)

        # Upscale fast using OpenCV (NOT EDSR)
        up = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)

        if last_results:
            for box in last_results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                x1 *= scale; y1 *= scale
                x2 *= scale; y2 *= scale

                cv2.rectangle(up, (x1, y1), (x2, y2), (0, 255, 0), 2)

        out.write(up)

    cap.release()
    out.release()


In [5]:
import cv2
import os

def diagnose_video_pipeline(video_path):
    issues = []

    # Check video existence
    if not os.path.exists(video_path):
        return ["Video file not found"]

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return ["OpenCV cannot open the video"]

    fps = cap.get(cv2.CAP_PROP_FPS)
    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    if fps <= 0:
        issues.append("⚠ FPS is invalid")
    if w <= 0 or h <= 0:
        issues.append("⚠ Frame dimensions invalid")

    ret, frame = cap.read()
    if not ret:
        issues.append("Cannot read frames")
    else:
        if frame is None:
            issues.append("Frame is None")
        if frame.ndim != 3:
            issues.append("Frame is not color (BGR)")

    cap.release()

    if not hasattr(cv2, "dnn_superres"):
        issues.append(" opencv-contrib-python not installed")

    return issues if issues else [" Video pipeline looks healthy"]


In [6]:
issues = diagnose_video_pipeline("Video.mp4")
for i in issues:
    print(i)
    issues = []

issues = diagnose_video_pipeline("Video1.mp4")
for i in issues:
    print(i)
    issues = []

issues = diagnose_video_pipeline("Video3.mp4")
for i in issues:
    print(i)
    issues = []

issues = diagnose_video_pipeline("Video3.mp4")
for i in issues:
    print(i)
    issues = []

 Video pipeline looks healthy
 Video pipeline looks healthy
 Video pipeline looks healthy
 Video pipeline looks healthy


In [7]:
import os

model_path = "sr_models/EDSR_x4.pb"

if not os.path.exists(model_path):
    print(f"Error: File does not exist at {model_path}")
else:
    file_size = os.path.getsize(model_path)
    print(f"File size: {file_size / (1024*1024):.2f} MB")
    
    if file_size < 100000: # If less than 100KB, it's likely HTML
        print("Error: The file is too small. You likely downloaded an HTML page instead of the model.")
        with open(model_path, 'r', errors='ignore') as f:
            first_line = f.readline()
            if "<!DOCTYPE html>" in first_line or "<html>" in first_line:
                print("   Confirmation: This is definitely an HTML file!")

File size: 36.79 MB


In [8]:
import os
import requests

def download_sr_model(model_name="EDSR_x4.pb"):
    # Ensure directory exists
    os.makedirs("sr_models", exist_ok=True)
    
    url = f"https://github.com/Saafke/EDSR_TensorFlow/raw/master/models/{model_name}"
    save_path = os.path.join("sr_models", model_name)
    
    print(f"Downloading {model_name} (this may take a minute)...")
    
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        file_size = os.path.getsize(save_path)
        print(f"Success! File saved to: {save_path}")
        print(f"Verified File Size: {file_size / (1024*1024):.2f} MB")
    else:
        print(f"Failed to download. Status code: {response.status_code}")

# Run the download
download_sr_model("EDSR_x4.pb")

Success! File saved to: sr_models\EDSR_x4.pb
Verified File Size: 36.79 MB


In [9]:
import cv2
from ultralytics import YOLO
from collections import Counter

# Internal reasoning map (SYSTEM LOGIC, not user input)
ISSUE_MAPPING = {
    "Damaged concrete structures": (
        "Structural damage in public infrastructure",
        "Infrastructure damage monitoring system"
    ),
    "DamagedElectricalPoles": (
        "Electrical pole damage posing public safety risk",
        "Electrical infrastructure monitoring system"
    ),
    "DamagedRoadSigns": (
        "Damaged or missing road signs affecting traffic safety",
        "Traffic safety monitoring system"
    ),
    "DeadAnimalsPollution": (
        "Dead animals causing hygiene and pollution issues",
        "Sanitation and animal welfare system"
    ),
    "FallenTrees": (
        "Fallen trees obstructing roads or walkways",
        "Disaster response and urban safety system"
    ),
    "Garbage": (
        "Improper garbage disposal in public areas",
        "Smart waste management system"
    ),
    "Graffitti": (
        "Vandalism detected on public property",
        "Urban maintenance and vandalism detection system"
    ),
    "IllegalParking": (
        "Vehicles illegally parked causing obstruction",
        "Traffic enforcement and parking monitoring system"
    ),
    "Potholes and RoadCracks": (
        "Road surface damage affecting commuter safety",
        "Road condition and maintenance monitoring system"
    )
}

def analyze_issue_from_video(video_path, yolo_model_path, frame_skip=5, conf=0.4):
    model = YOLO(yolo_model_path)
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise RuntimeError("Cannot open video")

    detected_classes = []
    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        if frame_id % frame_skip != 0:
            continue

        results = model(frame, conf=conf, verbose=False)

        for box in results[0].boxes:
            cls_id = int(box.cls[0])
            class_name = model.names[cls_id]   
            detected_classes.append(class_name)

    cap.release()

    if not detected_classes:
        return {
            "issue_detected": "No visible civic issue detected",
            "built_for": "Urban monitoring system",
            "confidence": 0.0
        }

    counts = Counter(detected_classes)
    dominant_class, freq = counts.most_common(1)[0]

    issue, purpose = ISSUE_MAPPING.get(
        dominant_class,
        ("General civic issue detected", "Civic issue monitoring system")
    )

    confidence = round(freq / sum(counts.values()), 2)

    return {
        "issue_detected": issue,
        "built_for": purpose,
        "dominant_issue_class": dominant_class,
        "confidence": confidence,
        "class_distribution": dict(counts)
    }


In [10]:
fast_video_yolo_sr(
    "video3.mp4",
    "output_video3.mp4",
    "runs/detect/train20/weights/best.pt",
    scale=2,
    skip_frames=2
)

d:\Projects\SIH\.venv\Lib\site-packages\ultralytics\nn\tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


In [11]:
analyze_issue_from_video("output_video2.mp4", "runs/detect/train20/weights/best.pt", frame_skip=2, conf=0.4)

d:\Projects\SIH\.venv\Lib\site-packages\ultralytics\nn\tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


{'issue_detected': 'Electrical pole damage posing public safety risk',
 'built_for': 'Electrical infrastructure monitoring system',
 'dominant_issue_class': 'DamagedElectricalPoles',
 'confidence': 0.64,
 'class_distribution': {'DamagedElectricalPoles': 58,
  'Graffitti': 13,
  'Garbage': 10,
  'FallenTrees': 1,
  'IllegalParking': 8}}

### CodeSnap — analyze_issue_from_video call

```python
analyze_issue_from_video("output_video2.mp4", "runs/detect/train20/weights/best.pt", frame_skip=2, conf=0.4)
```

- **Purpose:** Run YOLO detection over an output video and summarize the dominant civic issue.
- **Notes:** Ensure the function definitions and model path are available before running this cell.
- **Run:** Execute the preceding cells (functions/imports) then run this cell to get the analysis result.